<a href="https://colab.research.google.com/github/wooookim/ML-DL/blob/main/%EC%84%9C%EC%9A%B8%EC%8B%9C_%ED%8F%89%EA%B7%A0_%EA%B8%B0%EC%98%A8_%EC%98%88%EC%B8%A1_28_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np
from datetime import datetime

data = pd.read_csv('/content/train.csv')
submission = pd.read_csv('/content/sample_submission.csv')

In [4]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23011 entries, 0 to 23010
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   일시      23011 non-null  object 
 1   최고기온    23008 non-null  float64
 2   최저기온    23008 non-null  float64
 3   일교차     23007 non-null  float64
 4   강수량     9150 non-null   float64
 5   평균습도    23011 non-null  float64
 6   평균풍속    23007 non-null  float64
 7   일조합     22893 non-null  float64
 8   일사합     18149 non-null  float64
 9   일조율     22645 non-null  float64
 10  평균기온    23011 non-null  float64
dtypes: float64(10), object(1)
memory usage: 1.9+ MB
None


In [5]:
# '일시' 열을 날짜 데이터로 변환
data['일시'] = pd.to_datetime(data['일시'])

# 2월 29일에 해당하는 행 선택
selected_rows = data[(data['일시'].dt.month == 2) & (data['일시'].dt.day == 29)]

data = data.drop(selected_rows.index)

In [ ]:
data.info()

In [ ]:
data.head()

In [9]:
data['일시'].dtype

dtype('<M8[ns]')

In [15]:
# 계절 분류 함수 정의
def classify_season(month):
    if  month == 1 or month == 2 or month == 12:
        return '겨울'
    elif 3 <= month <= 5:
        return '봄'
    elif 6 <= month <= 8:
        return '여름'
    else:
        return '가을'

# '일시' 컬럼에서 월 정보를 추출하여 '계절' 열 추가
data['월'] = data['일시'].dt.month
data['계절'] = data['월'].apply(classify_season)

# 계절별 중앙값 계산
seasonal_medians = data.groupby('계절').median()

<ipython-input-15-d8fe624a71cf>:17: FutureWarning: The default value of numeric_only in DataFrameGroupBy.median is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  seasonal_medians = data.groupby('계절').median()


In [19]:
seasonal_medians

,최고기온,최저기온,일교차,강수량,평균습도,평균풍속,일조합,일사합,일조율,평균기온,월
계절,,,,,,,,,,,
가을,20.3,10.6,9.2,1.8,66.1,1.9,6.9,10.400,61.65,15.1,10.0
겨울,3.3,-4.6,7.8,0.3,60.4,2.2,6.4,8.000,63.75,-0.8,2.0
봄,17.5,7.6,10.0,1.4,58.9,2.6,8.1,16.440,62.60,12.3,4.0
여름,28.8,21.1,7.8,3.5,75.5,2.1,4.9,14.175,33.80,24.4,7.0


In [20]:
jan = data[(data['일시'].dt.month == 1)]
apr = data[(data['일시'].dt.month == 4)]
jul = data[(data['일시'].dt.month == 7)]
oct = data[(data['일시'].dt.month == 10)]

In [22]:
import numpy as np

# 각 달의 중앙값 계산
jan_median = jan['평균기온'].median()
apr_median = apr['평균기온'].median()
jul_median = jul['평균기온'].median()
oct_median = oct['평균기온'].median()

# 중앙값 대체 함수 정의
def replace_with_median(df, median_value):
    df['평균기온'] = np.where((df['평균기온'] - median_value) > 4, median_value, df['평균기온'])

# 중앙값으로 대체
replace_with_median(jan, jan_median)
replace_with_median(apr, apr_median)
replace_with_median(jul, jul_median)
replace_with_median(oct, oct_median)

<ipython-input-22-eab245c44d2a>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['평균기온'] = np.where((df['평균기온'] - median_value) > 4, median_value, df['평균기온'])


In [24]:
train = data[['최고기온', '최저기온', '일교차', '강수량', '평균습도', '평균풍속', '일조합', '일사합', '일조율']]
target = data['평균기온']

In [25]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures()
poly.fit(train)
train_poly = poly.transform(train)

from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(train_poly, target)


from sklearn.metrics import mean_absolute_error
lr_pred = lr.predict(train_poly)

lr_mae = mean_absolute_error(target, lr_pred)

print("훈련 세트 MAE:", lr_mae)

submission['평균기온'] = lr_pred[:358]
print(submission.head(30))

ValueError: ignored

In [ ]:
submission.to_csv("DEAGYUL_9_submission.csv", index=False)

print(submission.head(30))